In [30]:
import mujoco
import numpy as np
import pinocchio as pin
from robot_descriptions.skydio_x2_description import URDF_PATH
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

from mujoco_sysid import parameters
from mujoco_sysid.utils import muj2pin

model = mujoco.MjModel.from_xml_path(MJCF_PATH[: -len("x2.xml")] + "scene.xml")
mjdata = mujoco.MjData(model)
renderer = mujoco.Renderer(model, height=480, width=640)

for act_id in range(4):
    model.actuator(act_id).ctrlrange = np.array([-1e4, 1e4])

model.opt.timestep = 1e-3

In [31]:
sparceM = np.zeros((6, 6))

mujoco.mj_step(model, mjdata)
mujoco.mj_fullM(model, sparceM, mjdata.qM)

mSrc = sparceM[3:6, :3]
mrc = np.array([mSrc[2, 1], mSrc[0, 2], mSrc[1, 0]])
Ib = sparceM[3:6, 3:6]
vechIb = np.array([Ib[0, 0], Ib[0, 1], Ib[1, 1], Ib[0, 2], Ib[1, 2], Ib[2, 2]])
skydio_parameters = np.array([sparceM[0, 0], *mrc, *vechIb])

with np.printoptions(precision=5, suppress=True):
    print(skydio_parameters)

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [32]:
with np.printoptions(precision=5, suppress=True):
    print(parameters.get_dynamic_parameters(model, 1))

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [33]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)

with np.printoptions(precision=5, suppress=True):
    print(pinmodel.inertias[1].toDynamicParameters())

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [34]:
def mj_bodyRegressor(mj_model: mujoco.MjModel, mj_data: mujoco.MjData, body_id: int) -> np.ndarray:
    """
    Computes the body regressor matrix for a specific body in a MuJoCo model.

    Args:
        mj_model (mujoco.MjModel): MuJoCo model.
        mj_data (mujoco.MjData): MuJoCo data.
        body_id (int): ID of the body for which to compute the regressor matrix.

    Returns:
        np.ndarray: Body regressor matrix of shape (6, 10).
    """
    velocity = np.zeros(6)
    accel = np.zeros(6)
    _cross = np.zeros(3)

    mujoco.mj_objectVelocity(mj_model, mj_data, 2, body_id, velocity, 1)
    mujoco.mj_rnePostConstraint(mj_model, mj_data)
    mujoco.mj_objectAcceleration(mj_model, mj_data, 2, body_id, accel, 1)

    v, w = velocity[3:], velocity[:3]
    dv, dw = accel[3:], accel[:3]  # dv - classical acceleration, already contains g
    mujoco.mju_cross(_cross, w, v)
    dv -= _cross

    v1, v2, v3 = v
    v4, v5, v6 = w

    a1, a2, a3 = dv
    a4, a5, a6 = dw

    # fmt: off
    Y = np.array(
        [
            [a1 - v2 * v6 + v3 * v5, -(v5**2) - v6**2, -a6 + v4 * v5, a5 + v4 * v6, 0, 0, 0, 0, 0, 0],
            [a2 + v1 * v6 - v3 * v4, a6 + v4 * v5, -(v4**2) - v6**2, -a4 + v5 * v6, 0, 0, 0, 0, 0, 0],
            [a3 - v1 * v5 + v2 * v4, -a5 + v4 * v6, a4 + v5 * v6, -(v4**2) - v5**2, 0, 0, 0, 0, 0, 0],
            [0, 0, a3 - v1 * v5 + v2 * v4, -a2 - v1 * v6 + v3 * v4, a4, a5 - v4 * v6, -v5 * v6, a6 + v4 * v5, v5**2 - v6**2, v5 * v6],
            [0, -a3 + v1 * v5 - v2 * v4, 0, a1 - v2 * v6 + v3 * v5, v4 * v6, a4 + v5 * v6, a5, -(v4**2) + v6**2, a6 - v4 * v5, -v4 * v6],
            [0, a2 + v1 * v6 - v3 * v4, -a1 + v2 * v6 - v3 * v5, 0, -v4 * v5, v4**2 - v5**2, v4 * v5, a4 - v5 * v6, a5 + v4 * v6, a6],
        ]
    )
    # fmt: on

    return Y

In [35]:
import json

with open("../data/ltv_lqr_traj.json") as f:
    data = json.load(f)

In [36]:
np.random.seed(0)

# traj_idx = 2

# q = data["q"][traj_idx]
# v = data["v"][traj_idx]
# a = data["dv"][traj_idx]
# ctrl = data["u"][traj_idx]

q = np.random.randn(model.nq)
q[3:7] = q[3:7] / np.linalg.norm(q[3:7])
v = np.random.randn(model.nv)
a = np.random.randn(model.nv)
ctrl = np.random.randn(model.nu)

mjdata = mujoco.MjData(model)
mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = a
mjdata.ctrl[:] = ctrl

mujoco.mj_step(model, mjdata)
# mujoco.mj_forward(model, mjdata)
mujoco.mj_inverse(model, mjdata)

In [37]:
regressor = mj_bodyRegressor(model, mjdata, 1)

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(regressor)

[[  1.45259  -2.61468   1.19016 -26.9294    0.        0.        0.        0.        0.        0.     ]
 [ -0.62878  -0.80892  -0.5955   11.57007   0.        0.        0.        0.        0.        0.     ]
 [ -1.31113  27.13241  -9.40055  -2.05486   0.        0.        0.        0.        0.        0.     ]
 [  0.        0.       -1.31113   0.62878 -10.48531 -27.13241  -1.08476  -0.80892   1.45936   1.08476]
 [  0.        1.31113   0.        1.45259   0.10151  -9.40055 -27.0309    0.55982  -1.19016  -0.10151]
 [  0.       -0.62878  -1.45259   0.       -0.19062  -2.01918   0.19062 -11.57007 -26.9294   -0.99954]]


In [38]:
from mujoco_sysid.regressors import joint_body_regressor

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(joint_body_regressor(model, mjdata, 1))

[[  1.45259  -2.61468   1.19016 -26.9294    0.        0.        0.        0.        0.        0.     ]
 [ -0.62878  -0.80892  -0.5955   11.57007   0.        0.        0.        0.        0.        0.     ]
 [ -1.31113  27.13241  -9.40055  -2.05486   0.        0.        0.        0.        0.        0.     ]
 [  0.        0.       -1.31113   0.62878 -10.48531 -27.13241  -1.08476  -0.80892   1.45936   1.08476]
 [  0.        1.31113   0.        1.45259   0.10151  -9.40055 -27.0309    0.55982  -1.19016  -0.10151]
 [  0.       -0.62878  -1.45259   0.       -0.19062  -2.01918   0.19062 -11.57007 -26.9294   -0.99954]]


In [39]:
force = mjdata.qfrc_actuator.copy()
force[:3] = mjdata.xmat[1].reshape(3, 3).T @ force[:3]

with np.printoptions(precision=5, suppress=True):
    print(force)

[ 0.0027  -0.00025 -1.88903 -0.34375 -0.68952 -0.03839]


In [40]:
with np.printoptions(precision=5, suppress=True):
    print(regressor @ skydio_parameters)

[-0.00077 -0.00588 -1.88417 -0.3442  -0.69054 -0.03835]


### Set same values for pinocchio and compare

In [41]:
pinq, pinv = muj2pin(mjdata.qpos, mjdata.qvel)

pinq, pinv

(array([ 1.7639025 ,  0.40005527,  0.9791393 ,  0.57974326, -0.30322521,
         0.29577537,  0.69603819]),
 array([0.20482907, 0.3506959 , 0.17016347, 0.13355826, 1.4272426 ,
        0.76003818]))

In [42]:
pindata = pin.Data(pinmodel)
tau = pin.rnea(pinmodel, pindata, pinq, pinv, mjdata.qacc.copy())

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(pin.jointBodyRegressor(pinmodel, pindata, 1))

[[  8.99664  -2.61468   1.19016 -26.9294    0.        0.        0.        0.        0.        0.     ]
 [  7.57504  -0.80892  -0.5955   11.57007   0.        0.        0.        0.        0.        0.     ]
 [ -8.11583  27.13241  -9.40055  -2.05486   0.        0.        0.        0.        0.        0.     ]
 [  0.        0.       -8.11583  -7.57504 -10.48531 -27.13241  -1.08476  -0.80892   1.45936   1.08476]
 [  0.        8.11583   0.        8.99664   0.10151  -9.40055 -27.0309    0.55982  -1.19016  -0.10151]
 [  0.        7.57504  -8.99664   0.       -0.19062  -2.01918   0.19062 -11.57007 -26.9294   -0.99954]]


In [43]:
with np.printoptions(precision=5, suppress=True, linewidth=400):
    print("regressor", pin.jointBodyRegressor(pinmodel, pindata, 1) @ pinmodel.inertias[1].toDynamicParameters())
    print("rnea", tau)

regressor [  9.9951   10.86419 -10.90039  -0.93077  -0.15114  -0.03835]
rnea [  9.9951   10.86419 -10.90039  -0.93077  -0.15114  -0.03835]
